In [2]:
# download_ojp_grids.py
# -*- coding: utf-8 -*-
"""
下载 OJP 区域所需的栅格数据：
1. IGPP Earth free-air anomaly (FAA) 01m
2. GEBCO 01m bathymetry / topography

输出：
- E:\wjy\Gravity\data\1min_GravityData\faa_01m_150E-174E_10S-6N.nc
- E:\wjy\Gravity\data\1min_GravityData\gebco_01m_150E-174E_10S-6N.nc
"""

import os
import pygmt
import xarray as xr


# ================== 参数 ==================
REGION     = [111, 117, 14.5, 18.5]   # [lon_min, lon_max, lat_min, lat_max]
RESOLUTION = "01m"                # 1 arc-minute
DATA_DIR   = r"E:\wjy\Gravity\SCS_Gravity\data\Download_Data"

FAA_OUT    = os.path.join(DATA_DIR, "faa_01m_111E-117E_14.5N-18.5N.nc")
BATHY_OUT  = os.path.join(DATA_DIR, "gebco_01m_111E-117E_14.5N-18.5N.nc")


def main() -> None:
    os.makedirs(DATA_DIR, exist_ok=True)

    # ---------- 1. 下载 FAA ----------
    print("Downloading IGPP Earth free-air anomaly (FAA)...")
    faa = pygmt.datasets.load_earth_free_air_anomaly(
        resolution=RESOLUTION,
        region=REGION,
    )

    # 统一变量名与排序
    faa = faa.rename("z").sortby(["lat", "lon"])
    faa_ds = faa.to_dataset(name="z")
    faa_ds.to_netcdf(FAA_OUT)
    print(f"Saved FAA grid to: {FAA_OUT}")

    # ---------- 2. 下载 GEBCO 01m 地形 ----------
    print("Downloading GEBCO 01m bathymetry/topography...")
    bathy = pygmt.datasets.load_earth_relief(
        resolution=RESOLUTION,
        region=REGION,
        registration="gridline",
        data_source="gebco",  # 使用 GEBCO 数据源
    )

    bathy = bathy.rename("z").sortby(["lat", "lon"])
    bathy_ds = bathy.to_dataset(name="z")
    bathy_ds.to_netcdf(BATHY_OUT)
    print(f"Saved GEBCO bathymetry to: {BATHY_OUT}")

    print("All downloads finished.")


if __name__ == "__main__":
    main()


Saved FAA grid to: E:\wjy\Gravity\OJP_Gravity\data\Download_Data\faa_01m_111E-117E_14.5N-18.5N.nc
Saved GEBCO bathymetry to: E:\wjy\Gravity\OJP_Gravity\data\Download_Data\gebco_01m_111E-117E_14.5N-18.5N.nc
All downloads finished.


In [3]:
#下载GLOBSed数据
import xarray as xr, pygmt, numpy as np
from pathlib import Path

REGION = [106,122,9.5,23.5]
GLOBSED_PATH = r"E:\wjy\Gravity\SCS_Gravity\data\GlobSed\GlobSed\GlobSed_package3\GlobSed-v3.nc" 
OUT = Path(r"E:\wjy\Gravity\SCS_Gravity\data\GlobSed")

OUT.mkdir(parents=True, exist_ok=True)

# 1) 读取 GlobSed（单位：米，厚度为正）
sed = xr.open_dataset(GLOBSED_PATH).to_array().squeeze()  # 读出唯一变量
# 坐标名统一
if "longitude" in sed.coords: sed = sed.rename({"longitude":"lon"})
if "latitude"  in sed.coords: sed = sed.rename({"latitude":"lat"})

# 2) 区域裁剪（5′ 网格）
sed = pygmt.grdcut(sed, region=REGION).rename("z")

# 3) 对齐到 1′ gridline（与 GEBCO/CBA 对齐；注意仅为重采样，可比对齐，不提升原分辨率）
sed_01m = pygmt.grdsample(sed, region=REGION, spacing="1m", registration="gridline").rename("z")

# 4) 保存两份：原分辨率与 1′ 对齐版
sed.to_netcdf(OUT / "globsed_SCS_5min.nc")
sed_01m.to_netcdf(OUT / "globsed_SCS_01m_like.nc")

# 也导出三列 txt（lon lat thickness_m）
pygmt.grd2xyz(sed_01m, output_type="pandas").to_csv(
    OUT / "globsed_SCS_01m_like.txt", sep=" ", header=False, index=False, float_format="%.6f"
)

print("Done:", OUT)

Done: E:\wjy\Gravity\SCS_Gravity\data\GlobSed
